In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from IPython.display import display   #From stack overflow link below

#:https://stackoverflow.com/questions/51288869/print-visually-pleasing-dataframes-in-for-loop-in-jupyter-notebook-pandas

In [2]:
test = pd.read_csv('./datasets/test.csv')

# <span style = 'color: blue' > Cleaning </span>

**Viewing the DataFrame**

# Reformatting Columns and Column Names#

**Creating `test_cols` Variable with all the Dataframe Columns:**

In [3]:
test_cols = test.columns

**Making the headers all lower_case**


In [4]:
test.columns = [cols.lower() for cols in test.columns]

**Replace spaces with underscores**


In [5]:
test.columns = [cols.lower().replace(" ", "_") for cols in test.columns]

**Changing the name of `totrms_abv_grd` to `tot_rooms_abv_grd`**


In [6]:
test.rename(columns = {'totrms_abvgrd': 'tot_rooms_abvgrd'}, inplace = True)

### Creating Consistency in Column Names

**Changes `area` to `sf`:**

In [7]:
test.columns = [cols.replace('area', 'sf') for cols in test.columns]

**Changes `qc` to `qu`:**

In [8]:
test.columns = [cols.replace('qc', 'qu') for cols in test.columns]

**Changes `qual` to `qu`:**

In [9]:
test.columns = [cols.replace('qual', 'qu') for cols in test.columns]

**Changes `year` to `yr`:**

In [10]:
test.columns = [cols.replace('year', 'yr') for cols in test.columns]

**Adding an underscore after `bsmt` when there wasn't one:**

In [11]:
test.columns = [cols.replace("bsmt", "bsmt_") for cols in test.columns]

**Making sure there's not double underscores when there was already an underscore from fixes made in the above cell:**

In [12]:
test.columns = [cols.replace("__", "_") for cols in test.columns]

**Updating the `test_cols` variable:**

In [13]:
test_cols = test.columns

**Filling Numeric Nulls with the Mean**

In [14]:
null_cols = test.columns[test.isnull().any()] #Adapted from stack overflow, see link below

null_df = test[null_cols]  #Creating a dataframe with just columns that have nulls 

num_nulls = null_df._get_numeric_data()  #Extracting Numeric Null Cols 

for col in num_nulls:
    test[col].fillna(test[col].mean(), inplace = True)

#Code for creating `null_cols` adapted from stack overflow:   #Adapted from stackoverflow.com     https://stackoverflow.com/questions/37366717/pandas-print-column-name-with-missing-values
    

**Assigning Numerical Value to Ordinal Values in Catergorical Columns:**

Excellent(Ex) : 5 

Average/Average (TA) : 4 

Good (Gd) : 3 

Fair (Fa) : 2

Poor (Po) : 1

In [15]:
quality_numbers = { 'Ex' : 5 , 'TA' : 4 , 'Gd' : 3 , 'Fa': 2 , 'Po': 1}
test.replace ({
    'heating_qu': quality_numbers, 
    'pool_qu': quality_numbers, 
    'garage_cond': quality_numbers, 
    'garage_qu': quality_numbers, 
    'fireplacequ': quality_numbers, 
    'kitchen_qu': quality_numbers, 
    'bsmt_cond' : quality_numbers, 
    'bsmt_qu': quality_numbers, 
    'exter_cond': quality_numbers, 
    'exter_qu' : quality_numbers
}, inplace=True)

**Creating a Numerical Columns Variable Called `num_cols`:**

In [16]:
num_cols = test._get_numeric_data()

In [18]:
# sig_cols = test[['overall_qual', 'gr_liv_area', 'total_bsmt_sf', 'garage_area', 'garage_cars', '1st_flr_sf', 'year_built', 'full_bath', 'tot_rooms_abvgrd']]

In [19]:
# sig_cols_w_nulls = ['garage_cars', 'garage_sf', 'total_bsmt_sf']

## **Preprocessing**

**Creating interaction columns**

In [20]:
test['garage_score'] = test['garage_sf'] * test['garage_cars'] * test['garage_yr_blt']
test['yr_score'] = test['yr_built'] * test['yr_remod/add'] * test['garage_yr_blt']
test['bsmt_score'] = test['total_bsmt_sf'] * test['bsmt_fin_sf_1'] 

In [21]:
sig_df = test[['fireplaces',
               'gr_liv_sf',
               'overall_qu',
               'mas_vnr_sf',
               'full_bath',
               'tot_rooms_abvgrd',
               '1st_flr_sf',
               'garage_score',
               'yr_score',
               'bsmt_score'
               ]]

In [22]:
sig_df[sig_df['garage_score'].isnull()]

,fireplaces,gr_liv_sf,overall_qu,mas_vnr_sf,full_bath,tot_rooms_abvgrd,1st_flr_sf,garage_score,yr_score,bsmt_score


In [23]:
sig_df.isnull().sum()

fireplaces          0
gr_liv_sf           0
overall_qu          0
mas_vnr_sf          0
full_bath           0
tot_rooms_abvgrd    0
1st_flr_sf          0
garage_score        0
yr_score            0
bsmt_score          0
dtype: int64

In [24]:
# sig_cols = ['overall_qu', 'gr_liv_sf', '1st_flr_sf', 'yr_built', 'full_bath', 'tot_rooms_abvgrd']

In [25]:
model_from_pickling = pickle.load(open('kag_sub', 'rb'))

In [26]:
X = sig_df

In [27]:
preds = model_from_pickling.predict(X)

In [28]:
index = test['id']

In [29]:
#From Pete Plass 

submission = pd.DataFrame()
submission['SalePrice'] = np.round(preds)
submission['Id'] = index
submission.set_index ('Id', inplace = True)
submission.head()

,SalePrice
Id,
2658,138373.0
2718,180081.0
2414,197610.0
1989,102887.0
625,192766.0


In [31]:
submission.to_csv('./datasets/Kag_Sub_Ames_Housing_2.csv')